# Imports des librairies et données

In [1]:
import pandas as pd
from joblib import load
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

In [2]:
data = pd.read_csv("data.csv").iloc[:,1:-1]

In [3]:
data.shape

(96096, 5)

In [4]:
data.head()

,recency_days,frequency,total_amount,review_score,delivery_time
0,1757.0,1.0,141.90,5.0,6.0
1,1760.0,1.0,27.19,4.0,3.0
2,2183.0,1.0,86.22,3.0,26.0
3,1967.0,1.0,43.62,4.0,20.0
4,1934.0,1.0,196.89,5.0,13.0


# Preprocessing des données

In [5]:
X = data.fillna(0).values

In [6]:
scaler = MinMaxScaler()

In [7]:
X_scaled = scaler.fit_transform(X)

# Import du modèle initial

In [8]:
kmeans = load('kmeans.h5')

In [9]:
kmeans

KMeans(n_clusters=4, random_state=42)

In [ ]:
print("Score de Silhouette du K-Means seul : ", silhouette_score(X_scaled, kmeans.labels_))

KeyboardInterrupt: ignored

# Construction et entraînement de l'auto-encoder

In [10]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

my_input = Input(shape=(5,))

In [11]:
x = Dense(128, activation='relu')(my_input)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

encoded = Dense(5, activation='relu')(x)

x = Dense(32, activation='relu')(encoded)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation = 'relu')(x)

decoded = Dense(5, activation = 'sigmoid')(x)

# autoencoder
autoencoder = Model(my_input, decoded)

#encoder - used for our dimention reduction
encoder = Model(my_input, encoded)

autoencoder.compile(optimizer= 'adam', loss='categorical_crossentropy')

In [ ]:
X_scaled.shape

In [12]:
autoencoder.fit(X_scaled, X_scaled, batch_size = 128, epochs = 25,  verbose = 1)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


751/751 [==============================] - 48s 63ms/step - loss: 141617824.0000
Epoch 2/25
751/751 [==============================] - 45s 60ms/step - loss: 51045040128.0000
Epoch 3/25
751/751 [==============================] - 46s 61ms/step - loss: 1142895673344.0000
Epoch 4/25
508/751 [===================>..........] - ETA: 15s - loss: 5419241046016.0000

KeyboardInterrupt: ignored

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.save('autoencoder.h5')

# Transformation des features

In [ ]:
feat = encoder.predict(X_scaled)

3003/3003 [==============================] - 16s 5ms/step


In [ ]:
feat.shape

# Entraînement du nouveau modèle

In [ ]:
ae_kmeans = kmeans.fit(feat)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
ae_kmeans.save("ae_kmeans.h5")

In [ ]:
print("Score de Silhouette du K-Means amélioré : ", silhouette_score(feat, ae_kmeans.labels_))